### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [7]:
print("==> 0. pdf 문서 로딩")
loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

==> 0. pdf 문서 로딩
  총 109페이지 로드 완료


In [8]:
print("==> 1. 문서 분할 설정")
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = pythontext_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

==> 1. 문서 분할 설정
  104개 청크 생성 완료
  평균 청크 길이: 722자


In [12]:
print("==> 2. 임베딩 모델 설정")
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)
vectorstore = FAISS.from_documents(chunks, pythonembeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

==> 2. 임베딩 모델 설정
 FAISS 벡터스토어 생성 완료 (104개 벡터)


In [16]:
print("===> 3. 검색기 설정")
pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)
print("  검색기 설정 완료")

===> 3. 검색기 설정
  검색기 설정 완료


In [17]:
print("===> 4. LLM 설정")
pythonllm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)
print("  LLM 설정 완료")

===> 4. LLM 설정
  LLM 설정 완료


In [18]:
print("===> 5. 법률 자문 프롬프트 작성")
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""

pythonprompt = PromptTemplate(
    input_variables=["context", "question"],
    template=pythonprompt_template
)
print("  프롬프트 설정 완료")

===> 5. 법률 자문 프롬프트 작성
  프롬프트 설정 완료


In [19]:
print("===> 6. QA 체인 생성")
pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=pythonretriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": pythonprompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)
print("  RAG 파이프라인 구축 완료!")

===> 6. QA 체인 생성
  RAG 파이프라인 구축 완료!


In [21]:
print("===> 7. 테스트 질문 작성")
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]
print("  테스트 질문 작성 완료")

===> 7. 테스트 질문 작성
  테스트 질문 작성 완료


In [30]:
print("===> 8. 분쟁 유형 분류 함수")
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인 게임"]
    elearning_keywords = ["강의", "온라인 교육", "이러닝", "수강", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"
print("  분쟁 유형 분류 함수 작성 완료")

===> 8. 분쟁 유형 분류 함수
  분쟁 유형 분류 함수 작성 완료


In [31]:
print("===> 9. 질문 및 답변 실행")
for i, question in enumerate(pythontest_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = pythonqa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    
    # 분쟁 유형 분류 적용
    dispute_type = classify_dispute_type(question)
    print(f" 분쟁 유형: {dispute_type}\n")

    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)

===> 9. 질문 및 답변 실행

【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
 분쟁 유형: 게임

온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다. 제시된 사례들을 바탕으로 답변을 구성하겠습니다.

1. **사례 기반 답변**:
   - 사례 1-가-1(2006)에서는 시스템 오류로 소멸된 아이템에 대해 복구를 요구했으나, 계정 명의자가 아니면 복구가 불가하다는 게임사의 약관에 따라 복구가 거부되었습니다.
   - 사례 1-가-1(2009)에서는 시스템 오류로 인한 게임머니 소실을 주장했으나, 당시 유사한 불만이 접수되지 않았고, 사용 내역상 문제가 없었기 때문에 시스템 오류로 보기 어렵다는 이유로 복구가 거부되었습니다.

2. **법적 근거 제시**:
   - 게임사의 약관은 계약의 일부분으로 간주되며, 약관에 명시된 내용에 따라 복구 여부가 결정될 수 있습니다. 따라서 약관의 내용을 확인하는 것이 중요합니다.

3. **판례 참조**:
   - 앞서 언급한 사례들에서 볼 수 있듯이, 시스템 오류로 인한 피해 복구는 게임사의 약관과 당시 상황에 대한 증거에 따라 결정됩니다. 특히, 계정 명의 문제나 시스템 오류의 입증 여부가 중요한 쟁점이 됩니다.

4. **실무 가이드**:
   - **약관 확인**: 게임사의 약관을 확인하여 시스템 오류로 인한 아이템 소실에 대한 복구 정책을 파악합니다.
   - **증거 수집**: 시스템 오류가 발생한 시점의 증거(스크린샷, 로그 기록 등)를 수집하여 오류 발생을 입증할 수 있는 자료를 준비합니다.
   - **공식적인 이의 제기**: 게임사에 공식적으로 이의 제기를 하고, 복구 요청을 문서화하여 제출합니다.
   - **소